### Note:

This is the code for running CompuCell3D model parameter scans on the cluster.

To use this code, you may need to adapt the model path and software paths.

In [1]:
import numpy as np
import sys
from datetime import datetime
import subprocess
import os

import pipe_util2

In [2]:
# set the CompuCell3D software path
cc3d_param_scan_path = "path/to/CC3D_4.2.5_ubuntu_18.04_64bit/paramScan.sh"
install_path = "path/to/CC3D_4.2.5_ubuntu_18.04_64bit/lib/site-packages/cc3d"

In [11]:
def parameter_txt(rep, param_folder, scan_num, 
                  cc3d_param_scan_path, input_path, output_path, install_dir
                 ):
    """
    Creating parameter files in designated folders
    that can be later called in sbatch array.

    rep - int, number of repeat for each parameter combinations
    param_folder - str, path to folder that save parameter files
    scan_num - int, indexing parameter scans
    cc3d_param_scan_path - int, CompuCell3D parameter scan script path
    input_path - str, path to model .cc3d file
    output_path - str, path to a folder that save simulation results
    install_dir - str, CompuCell3D software path

    return - int, the number of parameter files generated
    """
    param_folder = pipe_util2.folder_verify(param_folder)
    
    i = 0
    while i<rep*scan_num:
        
        j = 0
        while j<rep:
            output_path = pipe_util2.folder_verify(output_path)
            o_path = output_path+"rep%s/"%str(j).zfill(2)
            pipe_util2.create_folder(o_path)
            
            k = 0
            while k<scan_num:
                
                file_num = str(i)
                f = open(param_folder+file_num+".p", "w")

                line = "%s --input=%s --output-dir=%s --install-dir=%s"%(
                    cc3d_param_scan_path, input_path, o_path, install_dir
                )
                f.write(line)
                f.close()
                
                i = i+1
                k = k+1
            j = j+1
    return i-1

In [9]:
def sbatch_file_polarity_sim(
    param_folder, scr_folder, name = None):
    """
    This function is to generate the sbatch file,
    facilitating submitting simulation jobs to the cluster. 

    param_folder - str, path to folder for storing parameter files.
    scr_folder - str, path to folder for storing both script .sh files and the output .o files.
    name - str, simulation name
    """
    
    now = datetime.now()
    dt_string = now.strftime("%d-%m-%Y-%H%M")
    scr_folder = pipe_util2.folder_verify(scr_folder)
    param_folder = pipe_util2.folder_verify(param_folder)
    
    if name:
        ofile_name = scr_folder + dt_string + "polarity_sim_" + name + ".o"
        sfile_name = scr_folder + dt_string + "polarity_sim_" + name + ".sh"
        
    else:
        ofile_name = scr_folder + dt_string + "polarity_sim" + ".o"
        sfile_name = scr_folder + dt_string + "polarity_sim" + ".sh"
    
    ofile_name = os.path.abspath(ofile_name)
    sfile_name = os.path.abspath(sfile_name)
    
    contents = """#!/bin/bash
#SBATCH --account=user
#SBATCH --partition=dept_cpu
#SBATCH --cpus-per-task=1
#SBATCH --mem=4G
#SBATCH --output=%s%s_%s.o
#SBATCH --job-name=potts_scan

bash %s$SLURM_ARRAY_TASK_ID.p
    """%(scr_folder, "%A", "%a", param_folder)
    s = open(sfile_name, "w")
    s.write(contents)
    s.close()
    
    return sfile_name

# 12222021_force_fpp_scan

In [ ]:
rep = 50
scan_num = 231
param_folder = "path/to/param_folder"
pipe_util2.create_folder(param_folder)

script_folder = "path/to/script_folder"
pipe_util2.create_folder(script_folder)

out_dir = "path/to/simulation/output/folder/CC3D_data_12222021_force_fpp_scan/"
pipe_util2.create_folder(out_dir)

input_path = "path/to/model/12222021_force_fpp_scan/force_fpp_gl.cc3d"

jobmax_num = parameter_txt(rep, param_folder, scan_num, 
              cc3d_param_scan_path, input_path, out_dir, install_path)
sbatch_file = sbatch_file_polarity_sim(os.path.abspath(param_folder), os.path.abspath(script_folder))

batch_size = 800
if jobmax_num<batch_size:
    batch_size = jobmax_num+1
batch_size = "%"+"%s"%batch_size
cmd = "sbatch --array=[0-%s]%s %s"%(jobmax_num, batch_size, sbatch_file)
subprocess.call(cmd, shell=True) # submit sbtach job to cluster.
cmd